In [3]:
import pandas as pd
from pandasql import sqldf
import numpy as np 

In [ ]:
#Есть табличка (скачать можно тут (https://storage.googleapis.com/tests-grkhr/promo.csv))
#customer_id — айди юзера
#ts — время заказа
#promo_code_type — тип промокода

#Мы завлекаем пользователей промо-кодами. Разные механики ведут себя по-разному. 
#Нужно найти топ-3 механики (promo_code_type), которые хорошо влияют на ретеншн пользователей после привлечения. 
#Также можно поискать плохие механики.



In [ ]:
#Выводы:
# В краткосрочной перспективе ТОП-3:
# 1. other_fixed_5
# 2. sorry
# 3. 3_for_10_fixed

# В долгосрочной перспективе:
# 1. other_percentage_20
# 2. other_fixed_5
# 3. sorry


# Совсем нЦе работают:
# other_fixed_15
# other_fixed_25
# other_percentage_10
# 3_for_20_fixed

# Рассчеты ниже:

In [4]:
promo = pd.read_csv('https://storage.googleapis.com/tests-grkhr/promo.csv')

In [5]:
#Форматирование и преобразование типа для столбца ts
promo['ts'] = promo['ts'].astype('datetime64[ns]')
promo['ts'] = promo['ts'].dt.strftime('%Y-%m-%d')
promo['ts'] = promo['ts'].astype('datetime64[ns]')

In [90]:
#Получаю список промокодов и смотрю количество выборок каждого промокода
promocodes = promo['promo_code_type'].loc[promo['promo_code_type'].notna()].unique()
promo['promo_code_type'].value_counts()


3_for_10_fixed         3081
other_fixed_20         2130
3_for_50_percentage    1501
3_for_30_percentage     944
other_fixed_30          325
sorry                   199
other_fixed_10          101
other_fixed_50           81
other_fixed_5            40
other_percentage_20      30
other_fixed_25           29
other_percentage_30      27
3_for_15_fixed           19
other_percentage_10       7
2_for_20_fixed            5
3_for_20_fixed            3
other_fixed_15            2
Name: promo_code_type, dtype: int64

In [47]:
#Функция, которая принимает название промокода, формирует когорту и возвращает значения retention
def get_retention_of_cohort(cohortname):
    campagin = cohortname
    
    #Получаю использования промокодов, дату первого использования и список пользователей когорты
    promocode_uses = promo.loc[promo['promo_code_type'] == campagin]
    campagin_customers = promocode_uses.groupby('customer_id')['ts'].min() 
    
    #Размечаю нужных пользователей в изначальном датасете
    cohort = promo.merge(campagin_customers, on = 'customer_id', how='inner')
    cohort.rename(columns={'ts_x':'transaction', 'ts_y': 'activate_promocode'}, inplace=True)

    #Выбираю события после активации промокода и считаю разницу между активацией и событием
    cohort = cohort.loc[cohort['transaction'] >= cohort['activate_promocode']]
    cohort['day_after'] = (cohort['transaction'] - cohort['activate_promocode']).dt.days

    #Косметические преобразования
    cohort = cohort.loc[cohort['day_after'] <= 60]
    cohort['cohort_name'] = cohortname

    #Расчет и формирование Retention
    retention = cohort.pivot_table(index='cohort_name', columns='day_after', values='customer_id', aggfunc=pd.Series.nunique)
    retention /= len(cohort['customer_id'].unique())
    retention = retention.round(3)
    return retention
    


In [59]:
#Рассчет и получения значений retention для каждого промокода.
retentions = pd.DataFrame()
for promocode in promocodes:
    retention = get_retention_of_cohort(promocode)
    retentions = retentions.append(retention)

retentions
    

day_after,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
cohort_name,,,,,,,,,,,,,,,,,,,,,
3_for_50_percentage,1.0,0.053,0.051,0.066,0.049,0.053,0.060,0.070,0.051,0.043,...,0.003,0.003,0.001,0.003,0.005,0.003,0.001,NaN,0.003,NaN
3_for_30_percentage,1.0,0.034,0.037,0.050,0.035,0.042,0.045,0.039,0.043,0.026,...,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN
3_for_10_fixed,1.0,0.063,0.067,0.073,0.062,0.061,0.074,0.075,0.046,0.048,...,0.022,0.026,0.020,0.018,0.019,0.018,0.021,0.018,0.016,0.022
other_fixed_15,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
other_fixed_50,1.0,0.048,NaN,0.016,0.032,NaN,0.016,NaN,0.016,NaN,...,0.016,0.032,NaN,NaN,0.032,0.032,0.016,0.016,NaN,0.016
other_fixed_30,1.0,0.013,0.013,0.010,0.010,0.010,0.003,0.003,0.006,0.010,...,0.006,0.013,0.019,0.019,0.006,0.010,0.006,0.003,0.006,NaN
other_fixed_20,1.0,0.018,0.015,0.018,0.017,0.022,0.019,0.020,0.019,0.022,...,0.015,0.021,0.013,0.023,0.012,0.018,0.017,0.014,0.017,0.017
other_fixed_10,1.0,0.033,0.022,0.043,0.054,0.043,NaN,0.022,0.054,NaN,...,0.022,0.033,0.043,0.022,0.011,0.054,0.022,0.011,0.033,0.033
other_percentage_30,1.0,0.077,0.038,0.115,0.192,0.077,NaN,0.154,0.038,0.077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
#Топ по 7-day Retention
seven_day_top = retentions[retentions[7].notna()].sort_values(by=7, ascending=False)
show_these_cohorts = [1, 2, 3]
seven_day_top.iloc[show_these_cohorts]

day_after,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
cohort_name,,,,,,,,,,,,,,,,,,,,,
other_fixed_5,1.0,0.091,0.030,NaN,0.091,0.030,0.152,0.121,0.061,0.182,...,0.061,0.091,0.091,0.091,0.091,0.091,0.091,0.061,NaN,0.030
sorry,1.0,0.090,0.084,0.084,0.090,0.112,0.112,0.118,0.118,0.084,...,0.039,0.045,0.028,0.028,0.022,0.045,0.034,0.017,0.011,0.034
3_for_10_fixed,1.0,0.063,0.067,0.073,0.062,0.061,0.074,0.075,0.046,0.048,...,0.022,0.026,0.020,0.018,0.019,0.018,0.021,0.018,0.016,0.022


In [95]:
#Топ по long-term Retention
long_term_top = retentions[retentions[50].notna()].sort_values(by=50, ascending=False)
show_these_cohorts = [1, 2, 3]
long_term_top.iloc[show_these_cohorts]

day_after,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
cohort_name,,,,,,,,,,,,,,,,,,,,,
other_percentage_20,1.0,0.067,0.100,0.033,NaN,0.067,0.033,0.067,NaN,NaN,...,NaN,0.067,0.033,0.067,0.067,0.067,0.067,0.100,0.067,0.067
other_fixed_5,1.0,0.091,0.030,NaN,0.091,0.030,0.152,0.121,0.061,0.182,...,0.061,0.091,0.091,0.091,0.091,0.091,0.091,0.061,NaN,0.030
sorry,1.0,0.090,0.084,0.084,0.090,0.112,0.112,0.118,0.118,0.084,...,0.039,0.045,0.028,0.028,0.022,0.045,0.034,0.017,0.011,0.034
